In [88]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [89]:
dataset = pd.read_csv('costsensitiveregression.csv')

In [90]:
dataset.head()

,NotCount,YesCount,ATPM,PFD,PFG,SFD,SFG,WP,WS,AH,AN,Status,FNC
0,2,21,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0
1,23,0,0.0,0.044,0.0,0.0,0.0,0.306179,0.0,0.0,0.0,1,0.0
2,1,22,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0
3,5,18,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,0.0
4,1,22,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0


In [91]:
dataset.columns

Index(['NotCount', 'YesCount', 'ATPM', 'PFD', 'PFG', 'SFD', 'SFG', 'WP', 'WS',
       'AH', 'AN', 'Status', 'FNC'],
      dtype='object')

In [92]:
X = dataset[['NotCount', 'YesCount', 'ATPM', 'PFD', 'PFG', 'SFD', 'SFG', 'WP', 'WS',
       'AH', 'AN']]
Y = dataset['Status']
FN = dataset['FNC']

In [93]:
X_ten = torch.tensor(X.values, dtype=torch.float32, requires_grad=True)
Y_ten = torch.tensor(Y.values, dtype=torch.float32, requires_grad=True)

TP_ten = torch.tensor([4 for i in range(len(Y_ten))], dtype=torch.float32, requires_grad=True)
FP_ten = torch.tensor([4 for i in range(len(TP_ten))], dtype=torch.float32, requires_grad=True)
TN_ten = torch.tensor([0 for i in range(len(TP_ten))], dtype=torch.float32, requires_grad=True)
FN_ten = torch.tensor(FN.values, dtype=torch.float32, requires_grad=True)


In [94]:
# performing test-train 80-20
indices = np.arange(len(dataset) , dtype = np.int64)
np.random.shuffle(indices)

train_indices = indices[:int(len(dataset)*0.8)]
test_indices = indices[int(len(dataset)*0.8):]

In [95]:
X_train = X_ten[train_indices]
Y_train = Y_ten[train_indices]
X_test = X_ten[test_indices]
Y_test = Y_ten[test_indices]

TP_train = TP_ten[train_indices]
TP_test = TP_ten[test_indices]
FP_train = FP_ten[train_indices]
FP_test = FP_ten[test_indices]
TN_train = TN_ten[train_indices]
TN_test = TN_ten[test_indices]
FN_train = FN_ten[train_indices]
FN_test = FN_ten[test_indices]

In [96]:
class CostSensitiveRegression(nn.Module):
    def __init__(self , input_dim , output_dim):
        super(CostSensitiveRegression, self).__init__()
        self.fc1 = nn.Linear(input_dim , output_dim)
        self.sigmoid1 = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid1(x)
        return x

In [97]:
def CostSensitiveLoss(y, y_hat, tp, fp, fn, tn):
    loss = torch.mean(y * (y_hat * tp + (1 - y_hat) * fn) + (1 - y) * (y_hat * fp + (1 - y_hat) * tn)).clone().detach()
    return  torch.tensor(loss, requires_grad=True)

In [98]:
epochs = 10
batch_size = 1000
num_batches = len(X_train) // batch_size + 1    # to account for the remainder of samples
tp = 4
fp = 4
tn = 0

model = CostSensitiveRegression(11 , 1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(epochs):

    for batch in range(num_batches):
        
        optimizer.zero_grad()
        x_batch = X_train[batch * batch_size : (batch + 1) * batch_size]
        y_batch = Y_train[batch * batch_size : (batch + 1) * batch_size]
        fn_batch = FN_train[batch * batch_size : (batch + 1) * batch_size]
        tn_batch = TN_train[batch * batch_size : (batch + 1) * batch_size]
        fp_batch = FP_train[batch * batch_size : (batch + 1) * batch_size]
        tp_batch = TP_train[batch * batch_size : (batch + 1) * batch_size]

        y_hat_probs = model(x_batch)
        y_hat = torch.tensor([1 if i > 0.5 else 0 for i in y_hat_probs])
        
        loss = CostSensitiveLoss(y_batch, y_hat, tp_batch , fp_batch , fn_batch , tn_batch)
        loss.backward()
        optimizer.step()

    y_hat_probs = model.forward(X_train)
    y_hat = torch.tensor([1 if i > 0.5 else 0 for i in y_hat_probs])
    loss = CostSensitiveLoss(Y_train, y_hat, TP_train , TP_train , FN_train , FN_train)
    print("Epoch: {} Loss: {}".format(epoch + 1, loss.item()))


C:\Users\tanay\AppData\Local\Temp\ipykernel_15164\1919002713.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return  torch.tensor(loss, requires_grad=True)


Epoch: 1 Loss: 539.3566284179688
Epoch: 2 Loss: 539.3566284179688
Epoch: 3 Loss: 539.3566284179688
Epoch: 4 Loss: 539.3566284179688
Epoch: 5 Loss: 539.3566284179688
Epoch: 6 Loss: 539.3566284179688
Epoch: 7 Loss: 539.3566284179688
Epoch: 8 Loss: 539.3566284179688
Epoch: 9 Loss: 539.3566284179688
Epoch: 10 Loss: 539.3566284179688
